# [Source](https://langchain-ai.github.io/langgraph/tutorials/get-started/3-add-memory/)

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
from langgraph.checkpoint.memory import InMemorySaver
from langchain_ollama import ChatOllama
from typing import Annotated
from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

In [2]:
llm = ChatOllama(
    model=os.getenv("ollama_model"),
    reasoning=True,
    model_kwargs={"think": "high"},
)

In [3]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [4]:
graph_builder = StateGraph(State)

In [5]:
tool = TavilySearch(max_results=2)
tools = [tool]
llm_with_tools = llm.bind_tools(tools)

In [6]:
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [7]:
graph_builder.add_node("chatbot", chatbot)

In [8]:
tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

In [9]:
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

In [10]:
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("tools", "chatbot")
graph = graph_builder.compile()

In [11]:
memory = InMemorySaver()

In [12]:
graph = graph_builder.compile(checkpointer=memory)

In [13]:
config = {"configurable": {"thread_id": "1"}}

In [14]:
user_input = "Hi there! My name is Max."

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Hi there! My name is Max.
================================== Ai Message ==================================

Hello, Max! 👋 How can I help you today?


In [15]:
user_input = "Remember my name?"

# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    config,
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Remember my name?
================================== Ai Message ==================================

Yes, I remember you’re Max! What can I help you with today?


In [16]:
# The only difference is we change the `thread_id` here to "2" instead of "1"
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]},
    {"configurable": {"thread_id": "2"}},
    stream_mode="values",
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Remember my name?
================================== Ai Message ==================================

I’m sorry—I don’t have a record of your name from our previous interactions. Could you let me know what you’d like me to call you?


In [17]:
snapshot = graph.get_state(config)
snapshot

StateSnapshot(values={'messages': [HumanMessage(content='Hi there! My name is Max.', additional_kwargs={}, response_metadata={}, id='85d29b2a-82ef-4b65-b123-d2e39d1244dd'), AIMessage(content='Hello, Max! 👋 How can I help you today?', additional_kwargs={'reasoning_content': 'User says: "Hi there! My name is Max." Simple greeting. Should respond friendly.'}, response_metadata={'model': 'gpt-oss:20b', 'created_at': '2025-09-27T11:24:31.23799758Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4515941585, 'load_duration': 3120225108, 'prompt_eval_count': 1342, 'prompt_eval_duration': 996123934, 'eval_count': 42, 'eval_duration': 398867173, 'model_name': 'gpt-oss:20b'}, id='run--5af9c5f3-4ed3-4104-8e2b-ecf6d9928169-0', usage_metadata={'input_tokens': 1342, 'output_tokens': 42, 'total_tokens': 1384}), HumanMessage(content='Remember my name?', additional_kwargs={}, response_metadata={}, id='5afce645-dceb-40ad-a02d-8e88c1caed49'), AIMessage(content='Yes, I remember you’re Max! What ca

In [18]:
snapshot.next  # (since the graph ended this turn, `next` is empty. If you fetch a state from within a graph invocation, next tells which node will execute next)

()